<a href="https://colab.research.google.com/github/Ayush-mishra-0-0/dav-project/blob/main/g20_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YouTube Comments Scrapping

In [ ]:
import pandas as pd
import numpy as np
from selenium import webdriver
import urllib
import pathlib
import time
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--incognito')
driver = webdriver.Chrome( options=chrome_options)

Making a function for getting the links from the web

In [ ]:
def get_links(item):
    query = urllib.parse.quote(item)
    url = "https://www.youtube.com/results?search_query=" + query
    driver.get(url)
    driver.execute_script("window.scrollTo(0, 1000000)")
    driver.maximize_window()
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a#video-title')))

    container = driver.find_elements(By.CSS_SELECTOR, 'a#video-title')
    titles = [element.get_attribute('title') for element in container]
    links = [element.get_attribute('href') for element in container]
    return list(zip(titles, links))
dfs = []
search_item = ["G20 2023","G-20","INDIA-G20","DELHI-SUMMIT 2023"]
for item in search_item:
    data = get_links(item)
    df = pd.DataFrame({'Item': [item] * len(data), 'Video Title': [d[0] for d in data], 'Links': [d[1] for d in data]})
    dfs.append(df)
final_df = pd.concat(dfs, ignore_index=True)
# final_df.to_csv('Links.csv', index=False)
final_df.shape

(89, 3)

In [ ]:
final_df=pd.read_csv('Links.csv')

In [ ]:
final_df.sample(10)

,Item,Video Title,Links
80,DELHI-SUMMIT 2023,G20 Summit 2023: US President Joe Biden to arr...,https://www.youtube.com/watch?v=P8qDdKKp8gY&pp...
49,INDIA-G20,"LIVE: Amid Israel-Hamas War, Mob Storms Russia...",https://www.youtube.com/watch?v=BVjtvIGDjm0&pp...
84,DELHI-SUMMIT 2023,G20 Summit 2023: US President Joe Biden Lands ...,https://www.youtube.com/watch?v=G4gqQgjNv0A&pp...
23,G-20,Highlights from the P20 Summit,https://www.youtube.com/watch?v=-boSXvXIDjc&pp...
50,INDIA-G20,Why Maldives' New President Wants Indian Troop...,https://www.youtube.com/watch?v=MuKPepZXuJE&pp...
29,G-20,G20 Summit Delhi: Bilateral meeting between PM...,https://www.youtube.com/shorts/oLYcl1MeiOA
19,G20 2023,Highlights Day-1 of the G20 Summit | Bharat Ma...,https://www.youtube.com/watch?v=GHsSldQROvo&pp...
46,INDIA-G20,G2O Summit 2023: India’s Modi chairs summit of...,https://www.youtube.com/watch?v=F1pUDSue8mQ&pp...
60,INDIA-G20,How India is clearing and hiding away slums as...,https://www.youtube.com/watch?v=Grspbh_1piA&pp...
78,DELHI-SUMMIT 2023,Sansad TV Special। North East India : Gateway ...,https://www.youtube.com/watch?v=iFyqgJsCELU&pp...


In [ ]:
null_counts = final_df.groupby('Item')['Links'].apply(lambda x: x.isnull().sum())
print(null_counts)

Item
DELHI-SUMMIT 2023    0
G-20                 0
G20 2023             0
INDIA-G20            0
Name: Links, dtype: int64


In [ ]:
final_df.dropna(inplace=True)
final_df.shape

(89, 3)

Defining the function to scrap the required information from the links

In [ ]:
videos_dictionary = {
    'Comments': {},
    'Video Link': {},
    'Video Title': {},
    'Item': {}
}
from urllib.parse import urlparse, parse_qs

def get_query_from_url(url):
    # Parse the URL and extract the query parameter
    parsed_url = urlparse(url)
    query = parse_qs(parsed_url.query).get('search_query', '')
    return query[0] if query else ''

def scrap(url, i):
    print('Fetched date and time - ', datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
    try:
        videos_dictionary['Video Link'][i] = url
    except Exception as e:
        print(f"Error updating Video Link for index {i}: {e}")

    driver.get(url)
    time.sleep(3)
    item= get_query_from_url(url)
    try:
        video_title = driver.find_element("xpath",'//*[@id="video-title"]').text
        videos_dictionary['Video Title'][i] = video_title
        videos_dictionary['Item'][i] = item
    except Exception as e:
        print(f"Error updating Video Title or Item for index {i}: {e}")

    # Scroll down to load comments
    comments = driver.find_element("xpath",'//*[@id="comments"]')
    # driver.find_element("xpath", '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[1]/div/div/div[2]/div/div[1]/div/div[1]/input')
    driver.execute_script("arguments[0].scrollIntoView();", comments)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    username_elems = driver.find_elements("xpath",'//*[@id="author-text"]')
    comment_elems = driver.find_elements("xpath",'//*[@id="content-text"]')

    comment_string = ''

    for user, comment in zip(username_elems, comment_elems):
        comment_string = comment_string + user.text + ' - ' + comment.text + '\n'

    try:
        videos_dictionary['Comments'][i] = comment_string
    except Exception as e:
        print(f"Error updating Comments for index {i}: {e}")



First check the file exist to save the data that has been scrapped from the links. If the file does not exist, then create one. If it exist then check whether it is empty or not.

If not empty, then extract data drom the links not present in the file.

In [ ]:
start = 0
name = 'Youtube_scrapping_comments.csv'
file_name = pathlib.Path(name)
if file_name.exists():
    print ("File exist.")
    try :
        print("Reading the file now")
        df_temp = pd.read_csv(name, index_col = 0)
        start = (len(df_temp) )
    except:
        print("File is empty")
else:
    print ("File does not exist\n", "Creating the file")
    file = open(name,"w+")
    print("File created successfully with filename - ",name)
print("Starting from position - ", start)

File exist.
Reading the file now
File is empty
Starting from position -  0


In [ ]:
for num in range(start, len(final_df)):
    url = final_df['Links'].iloc[num]
    print("Loop entered")
    try:
        print("getting link-", num)
        driver.get(url)
    except:
        print("Not getting")
        continue
    print("=" * 40)
    print("Scraping " + url)
    scrap(url, num)
    print("=" * 40)
print("Process ended successfully")


Loop entered
getting link- 0
Scraping https://www.youtube.com/watch?v=F1pUDSue8mQ&pp=ygUIRzIwIDIwMjM%3D
Fetched date and time -  11/11/2023 18:10:43
Loop entered
getting link- 1
Scraping https://www.youtube.com/watch?v=X8rM829c7to&pp=ygUIRzIwIDIwMjM%3D
Fetched date and time -  11/11/2023 18:10:57
Loop entered
getting link- 2
Scraping https://www.youtube.com/watch?v=dbgGV5-TrdE&pp=ygUIRzIwIDIwMjM%3D
Fetched date and time -  11/11/2023 18:11:05
Loop entered
getting link- 3
Scraping https://www.youtube.com/watch?v=G_O8ogCpbRI&pp=ygUIRzIwIDIwMjM%3D
Fetched date and time -  11/11/2023 18:13:28
Loop entered
getting link- 4
Scraping https://www.youtube.com/watch?v=Lf-hYTjapiw&pp=ygUIRzIwIDIwMjM%3D
Fetched date and time -  11/11/2023 18:13:51
Loop entered
getting link- 5
Scraping https://www.youtube.com/watch?v=0q20I1xvtSA&pp=ygUIRzIwIDIwMjM%3D
Fetched date and time -  11/11/2023 18:14:19
Loop entered
getting link- 6
Scraping https://www.youtube.com/watch?v=jY-MLj2liZQ&pp=ygUIRzIwIDIwMjM%3D
Fe

In [ ]:
data = pd.DataFrame.from_dict(videos_dictionary)
data

,Comments,Video Link,Video Title,Item
0,@mjfansumit - I proud that at this event I wor...,https://www.youtube.com/watch?v=F1pUDSue8mQ&pp...,World leaders arrive at the Bharat Mandapam in...,
1,@priyachattaraj9926 - I proud of you modi sir ...,https://www.youtube.com/watch?v=X8rM829c7to&pp...,PM Modi addresses The Hindustan Times Leadersh...,
2,- प्रश्न. G20 की स्थापना किस वर्ष हुई?\n\na. ...,https://www.youtube.com/watch?v=dbgGV5-TrdE&pp...,India's Big Win at G20 Summit: G20 New Delhi L...,
3,@mukundrajan8961 - The list of invitees / part...,https://www.youtube.com/watch?v=G_O8ogCpbRI&pp...,G-4 | G-6 | G-7 | G-8 | G-10 | G-11 | G-20 | D...,
4,@DhimanBharadwaj - Very much proud to see Bhar...,https://www.youtube.com/watch?v=Lf-hYTjapiw&pp...,Turkish Parliament Boycotts Coca Cola and Nest...,
...,...,...,...,...
84,@lamwilshangdhi612 - Diplomats are Tired from ...,https://www.youtube.com/watch?v=G4gqQgjNv0A&pp...,Why US Wants India to Play a Bigger Role in th...,
85,"@user-pj5hi8ee5p - Wow, finally Casro get to v...",https://www.youtube.com/watch?v=5AsWXHdjanA&pp...,How changing Centre-States relations is affect...,
86,@yashbkanungo6357 - N heart touching n beautif...,https://www.youtube.com/watch?v=F_afwCgKxQo&pp...,G20 Summit in delhi: दिल्ली में कैसी हैं जी-20...,
87,@unity.1 - I am a citizen of the Republic of ...,https://www.youtube.com/watch?v=LjqT1stKstE&pp...,Why US Wants India to Play a Bigger Role in th...,


In [ ]:
data.shape

(89, 4)

In [ ]:
filtered_data = data[data['Comments'] != '']
data = data[data['Comments'] != '']
data.reset_index(drop=True, inplace=True)

In [ ]:
data= pd.read_csv('Youtube_scrapping_comments.csv')

In [ ]:
driver.close()